In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm
import scipy
from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [2]:
X_train = pd.read_csv("data/train_inter.csv")
q_train = pd.read_csv("data/freq_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

f_train = pd.read_csv("data/train_features.csv", encoding = "ISO-8859-1")
f_train = f_train.ix[:, 2:]
X_train = pd.concat([X_train, f_train], axis=1)


In [26]:
X_test = pd.read_csv("data/test_inter.csv")
q_test = pd.read_csv("data/freq_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

f_test = pd.read_csv("data/test_features.csv", encoding = "ISO-8859-1")
f_test = f_test.ix[:, 2:]
X_test = pd.concat([X_test, f_test], axis=1)

In [3]:
X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))


X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())


In [27]:
X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [4]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../tmp/GoogleNews-vectors-negative300.bin', binary=True)

In [50]:
del word2vec

In [5]:
def to_vec(x):
    result = word2vec.word_vec('yes') - word2vec.word_vec('yes')
    co = 0
    for i in x:
        if i in word2vec.vocab:
            result += word2vec.word_vec(i)
            co += 1
    return result / co

In [6]:
def norm3(x, y):
    arr = []
    for i in x.index:
        arr += [abs(x.ix[i] - y.ix[i])]
    return arr

In [7]:
def norm2(x, y):
    arr = []
    for i in x.index:
        if i % 50000 == 0:
            print(i)
        arr += [scipy.spatial.distance.cosine(x.ix[i], y.ix[i])]
    return arr

In [8]:
from numpy import linalg as LA
def norm1(x, y):
    arr = []
    for i in x.index:
        if i % 50000 == 0:
            print(i)
        arr += [LA.norm(x.ix[i] - y.ix[i])]
    return arr

In [9]:
tmp1 = X_train['question1'].apply(lambda x: to_vec(x))
tmp2 = X_train['question2'].apply(lambda x: to_vec(x))
tmp1 = pd.DataFrame(list(tmp1))
tmp2 = pd.DataFrame(list(tmp2))
#X_train = pd.concat([X_train, pd.DataFrame(X_train['question1'].apply(lambda x: to_vec(x))),
#                  pd.DataFrame(X_train['question2'].apply(lambda x: to_vec(x)))], axis=1)
from numpy import linalg as LA
#del tmp1
#del tmp2

In [10]:
X_train = pd.concat([X_train, tmp1],axis=1)
X_train = pd.concat([X_train, tmp2],axis=1)

In [11]:
tmp1.fillna(tmp1.mean(), inplace=True)
tmp2.fillna(tmp2.mean(), inplace=True)
1

1

In [12]:
n1 = norm1(tmp1, tmp2)
n2 = norm2(tmp1, tmp2)
#n3 = norm3(tmp1, tmp2)

0
50000
100000
150000
200000
250000
300000
350000
400000
0
50000
100000
150000
200000
250000
300000
350000
400000


In [13]:
n1 = pd.DataFrame(n1)
n1.columns = ['n1']
n2 = pd.DataFrame(n2)
n2.columns = ['n2']
#n3 = pd.DataFrame(n3)
#n3.columns = ['n3']

In [14]:
X_train = pd.concat([X_train, n1],axis=1)
X_train = pd.concat([X_train, n2],axis=1)

In [15]:
del X_train['question1']
del X_train['question2']
del X_train['qid1']
del X_train['qid2']
del X_train['id']

#del X_test['question1']
#del X_test['question2']
#del X_test['test_id']

In [16]:
del X_train['n1']
del X_train['n2']

In [17]:
X_train = pd.concat([X_train, n1],axis=1)


In [18]:
X_train = pd.concat([X_train, n2],axis=1)

In [19]:
X_train[:2]

,word_match,tfidf_word_match,q1,q2,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,...,292,293,294,295,296,297,298,299,n1,n2
0,0.727273,0.772164,1.818125e-07,3.636250e-07,66,57,9,20,20,14,...,-0.080499,-0.015268,-0.058472,-0.016852,0.026965,0.056458,0.058258,-0.04502,0.238747,0.024187
1,0.307692,0.361758,1.454500e-06,5.454375e-07,51,88,-37,21,29,8,...,-0.050604,0.013800,0.014710,0.007458,-0.011377,-0.032227,0.067801,-0.09425,1.115507,0.455235


In [33]:
del f_test

In [43]:
import gc
gc.collect()

2733

In [35]:
tmp1 = X_test['question1'].apply(lambda x: to_vec(x))
tmp1 = pd.DataFrame(list(tmp1))

#X_train = pd.concat([X_train, pd.DataFrame(X_train['question1'].apply(lambda x: to_vec(x))),
#                  pd.DataFrame(X_train['question2'].apply(lambda x: to_vec(x)))], axis=1)
#from numpy import linalg as LA
#del tmp1
#del tmp2

In [37]:
X_test = pd.concat([X_test, tmp1],axis=1)

In [59]:
del tmp1

NameError: name 'tmp1' is not defined

In [40]:
tmp2 = X_test['question2'].apply(lambda x: to_vec(x))
tmp2 = pd.DataFrame(list(tmp2))

In [41]:
X_test = pd.concat([X_test, tmp2],axis=1)

In [58]:
del tmp2

NameError: name 'tmp2' is not defined

In [ ]:
tmp1.fillna(tmp1.mean(), inplace=True)
tmp2.fillna(tmp2.mean(), inplace=True)
1

In [ ]:
n1 = norm1(tmp1, tmp2)

In [ ]:
n2 = norm2(tmp1, tmp2)

In [38]:
n1 = pd.DataFrame(n1)
n1.columns = ['n1']
n2 = pd.DataFrame(n2)
n2.columns = ['n2']

In [40]:
n1.to_csv("data/n1.csv", index=None)
n2.to_csv("data/n2.csv", index=None)

In [44]:
n1 = pd.read_csv("data/n1.csv")
n2 = pd.read_csv("data/n2.csv")

In [45]:
del X_test['question1']
del X_test['question2']
del X_test['test_id']

In [57]:
del f_test

NameError: name 'f_test' is not defined

In [ ]:
X_test = pd.concat([X_test, n1],axis=1)
X_test = pd.concat([X_test, n2],axis=1)

In [46]:
X_test['n1'] = n1.values

In [47]:
X_test['n2'] = n2.values

In [48]:
X_test[:3]

,word_match,tfidf_word_match,q1,q2,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,...,292,293,294,295,296,297,298,299,n1,n2
0,0.266667,0.274019,1.818125e-07,1.818125e-07,57,68,-11,21,22,11,...,-0.090914,0.028786,-0.034929,-0.051687,0.034475,-0.023356,0.061010,-0.048594,0.939924,0.350222
1,0.500000,0.480962,3.636250e-07,3.636250e-07,66,43,23,24,19,14,...,-0.151204,0.042806,0.024628,0.166341,0.136393,0.009206,0.027445,-0.027832,0.794613,0.162770
2,0.444444,0.468893,1.818125e-07,1.818125e-07,60,29,31,21,15,14,...,-0.088867,-0.055176,0.035522,-0.014801,0.108215,-0.110840,-0.020203,-0.074707,0.964996,0.216529


In [62]:
X_test

,word_match,tfidf_word_match,q1,q2,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,...,292,293,294,295,296,297,298,299,n1,n2
0,0.266667,0.274019,1.818125e-07,1.818125e-07,57,68,-11,21,22,11,...,-0.090914,0.028786,-0.034929,-0.051687,0.034475,-0.023356,0.061010,-0.048594,0.939924,0.350222
1,0.500000,0.480962,3.636250e-07,3.636250e-07,66,43,23,24,19,14,...,-0.151204,0.042806,0.024628,0.166341,0.136393,0.009206,0.027445,-0.027832,0.794613,0.162770
2,0.444444,0.468893,1.818125e-07,1.818125e-07,60,29,31,21,15,14,...,-0.088867,-0.055176,0.035522,-0.014801,0.108215,-0.110840,-0.020203,-0.074707,0.964996,0.216529
3,0.000000,0.000000,1.818125e-07,1.818125e-07,27,17,10,16,13,4,...,-0.218994,0.140381,-0.099884,0.011475,0.126587,0.097412,0.327148,-0.124573,1.529496,0.340773
4,0.800000,1.000000,1.818125e-07,1.818125e-07,32,30,2,17,16,4,...,-0.126123,-0.046191,-0.076221,0.013788,0.018469,0.009265,-0.042554,-0.005212,1.097259,0.248382
5,0.200000,0.222582,1.818125e-07,1.818125e-07,60,55,5,20,18,10,...,-0.087173,-0.023031,-0.058567,0.025133,0.006897,0.039293,-0.021708,0.032661,1.035971,0.432418
6,0.600000,0.624099,5.090750e-06,5.454375e-06,45,49,-4,21,20,10,...,-0.075903,-0.016815,-0.111157,0.020350,-0.029187,0.004817,-0.077740,-0.037792,0.401771,0.048552
7,0.363636,0.285686,1.818125e-07,1.818125e-07,98,98,0,29,24,18,...,-0.068559,0.033026,0.027974,-0.009907,0.020446,-0.042525,0.007923,-0.048756,0.922094,0.392162
8,0.571429,0.588009,1.818125e-07,1.818125e-07,40,56,-16,16,16,9,...,-0.060417,-0.027512,-0.051453,-0.021095,0.022598,0.033279,0.080559,-0.007858,0.620887,0.150162
9,0.235294,0.220013,1.818125e-07,3.636250e-07,107,41,66,25,17,22,...,-0.131429,-0.088379,-0.036611,-0.025810,-0.029012,-0.069946,-0.028646,-0.042338,0.928094,0.268251


In [45]:
del X_test['n1']
del X_test['n2']

In [87]:
import gc
gc.collect()

0

In [73]:
del tmp1

NameError: name 'tmp1' is not defined

In [16]:
print(X_test.q1.mean())
print(X_train.q1.mean())


9.668771881958455e-07
9.009205147102744e-07


In [31]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = -1
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 1200

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.7,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 50,
    #'max_depth': 6
    #'num_class': 3
    
}

pred_train = np.zeros(len(y_train))
xgbs = []
sc,sc_mean = [],[]

for itr, ite in folds:
    ypred = []
    for i in range(1):
               
        param['seed'] = i+1
        
        #xgboost
        '''
        Xdatatrain = xgb.DMatrix(data=X_train.ix[itr, :].values,
                                     label=y_train.ix[itr].values)
        Xdataval = xgb.DMatrix(data=X_train.ix[ite, :].values,
                                     label=y_train.ix[ite].values)

        plst = list(param.items())
        watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
        bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=5)
        
        
        #rc = ensemble.ExtraTreesClassifier(n_estimators=1300, criterion='gini', max_depth=None, n_jobs=-1)
        #pred_train[ite] = bst.predict(Xdataval)
        #neigh = KNeighborsClassifier(n_neighbors=2)
        #neigh.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = bst.predict(Xdataval)
        #ypred = neigh.predict_proba(X_train.ix[ite, :])
        xgbs.append(bst)
        
        '''
        # train
        lgb = lgbm.sklearn.LGBMClassifier(n_estimators=300, seed=0, **params)
        lgb.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = lgb.predict_proba(X_train.ix[ite, :])[:, 1]

        '''
        model.fit(X_train.ix[itr, :].values, y_train_cat[itr],
            epochs=60,
            batch_size=1000)
        ypred = model.predict(X_train.ix[ite, :].values)
        '''
    #ypred = sum(ypred) / len(ypred) 0.401408 0.392476
    pred_train[ite] = ypred
    
    
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

XGB: 0.294 +- 0.001


In [103]:
def foo(x):
    a = 0.165 / 0.37
    b = (1 - 0.165) / (1 - 0.37) 
    return  a * x / (a * x + b * (1 - x))

In [33]:
pred_train = foo(pred_train)

In [34]:
pred_train = pd.DataFrame(pred_train)
pred_train.columns = ['y']
pred_train.to_csv("stacking/lgb_13.csv", index=None)

In [ ]:
1+1

### Тест

In [104]:
submit = pd.read_csv("data/sample_submission.csv")

In [21]:

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.7,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 50,
    #'max_depth': 6
    #'num_class': 3
    
}


In [25]:
lgb = lgbm.sklearn.LGBMClassifier(n_estimators=300, seed=0, **params)
lgb.fit(X_train, y_train)


/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=50,
        min_split_gain=0, n_estimators=300, nthread=-1, num_leaves=31,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [76]:
X_test.ix[:1, :]

,word_match,tfidf_word_match,q1,q2,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,...,292,293,294,295,296,297,298,299,n1,n2
0,0.266667,0.274019,1.818125e-07,1.818125e-07,57,68,-11,21,22,11,...,-0.090914,0.028786,-0.034929,-0.051687,0.034475,-0.023356,0.061010,-0.048594,0.939924,0.350222
1,0.500000,0.480962,3.636250e-07,3.636250e-07,66,43,23,24,19,14,...,-0.151204,0.042806,0.024628,0.166341,0.136393,0.009206,0.027445,-0.027832,0.794613,0.162770


In [88]:
test_pred1 = lgb.predict_proba(X_test.ix[1000000:, :])[:, 1]

In [78]:
test_pred = pd.DataFrame()

In [79]:
test_pred = test_pred1

In [90]:
test_pred[:4]

array([ 0.00476483,  0.5744925 ,  0.56159276,  0.00071559])

In [91]:
test_pred1[:4]

array([  1.17564273e-03,   5.03788965e-01,   3.26500719e-04,
         2.08484644e-03])

In [92]:
len(test_pred)

1000001

In [134]:
len(test_pred)

2345796

In [97]:
tmp = np.concatenate([test_pred, test_pred1], axis=0)

In [132]:
len(tmp)

2345797

In [135]:
tmp[1000000:]

array([  1.17564273e-03,   1.17564273e-03,   5.03788965e-01, ...,
         1.91245909e-04,   5.48407340e-01,   2.53529380e-01])

In [133]:
t1 = tmp[:1000000]
t2 = tmp[1000001:]
test_pred = np.concatenate([t1, t2], axis=0)

In [119]:
del test_pred[1000000]

ValueError: cannot delete array elements

In [136]:
test_pred[999999:]

array([  6.40756308e-01,   1.17564273e-03,   5.03788965e-01, ...,
         1.91245909e-04,   5.48407340e-01,   2.53529380e-01])

In [117]:
test_pred = tmp

In [137]:
len(test_pred)

2345796

In [15]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 1100

In [ ]:
Xdatatrain = xgb.DMatrix(data=X_train.ix[:, :].values,
                                     label=y_train.ix[:].values)
Xdataval = xgb.DMatrix(data=X_train.values,
                                     label=y_train.values)
Xdatatest = xgb.DMatrix(data=X_test.values)


plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
#bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


#test_pred = bst.predict(Xdatatest)



In [16]:
Xdatatrain = xgb.DMatrix(data=X_train.ix[:, :].values,
                                     label=y_train.ix[:].values)

In [17]:
Xdataval = xgb.DMatrix(data=X_train.ix[:10, :].values,
                                     label=y_train.ix[:10].values)

In [20]:
Xdatatest = xgb.DMatrix(data=X_test.ix[:, :].values)


In [21]:

plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]

In [22]:
bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


[0]	train-logloss:0.645145	eval-logloss:0.653016
[15]	train-logloss:0.394068	eval-logloss:0.437614
[30]	train-logloss:0.350926	eval-logloss:0.400341
[45]	train-logloss:0.337777	eval-logloss:0.401014
[60]	train-logloss:0.330209	eval-logloss:0.387795
[75]	train-logloss:0.324804	eval-logloss:0.387937
[90]	train-logloss:0.320158	eval-logloss:0.383231
[105]	train-logloss:0.316191	eval-logloss:0.384437
[120]	train-logloss:0.313145	eval-logloss:0.380075
[135]	train-logloss:0.309691	eval-logloss:0.37662
[150]	train-logloss:0.306753	eval-logloss:0.375377
[165]	train-logloss:0.304356	eval-logloss:0.370649
[180]	train-logloss:0.301772	eval-logloss:0.358448
[195]	train-logloss:0.299199	eval-logloss:0.355894
[210]	train-logloss:0.296768	eval-logloss:0.359679
[225]	train-logloss:0.294945	eval-logloss:0.36081
[240]	train-logloss:0.293169	eval-logloss:0.354961
[255]	train-logloss:0.291001	eval-logloss:0.353774
[270]	train-logloss:0.289022	eval-logloss:0.350644
[285]	train-logloss:0.28755	eval-logloss:

In [23]:
test_pred = bst.predict(Xdatatest)


In [138]:
test_pred = foo(test_pred)

In [139]:
len(submit)

2345796

In [140]:
submit['is_duplicate'] = test_pred

In [141]:
submit.to_csv("stacking/lgb_13_test.csv", index=None)

In [143]:
submit

,test_id,is_duplicate
0,0,0.001608
1,1,0.312369
2,2,0.301189
3,3,0.000241
4,4,0.172144
5,5,0.001559
6,6,0.965719
7,7,0.077142
8,8,0.422686
9,9,0.002348


In [59]:
submit

,test_id,is_duplicate
0,0,0.004466
1,1,0.235243
2,2,0.166915
3,3,0.000614
4,4,0.457306
5,5,0.001726
6,6,0.964534
7,7,0.138894
8,8,0.414888
9,9,0.004965
